In [2]:
import pandas as pd                                                   
from tensorflow.keras.models import Sequential                        
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU               
from sklearn.preprocessing import MinMaxScaler                        
import numpy as np                                                    
import matplotlib.pyplot as plt                                        
import pickle
from scipy.stats import pearsonr
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.dataprocessing.transformers import Scaler
from darts import TimeSeries
from darts.dataprocessing.transformers import MinTReconciliator
from darts.utils.likelihood_models import GaussianLikelihood

In [3]:
cal_day = []
cal_month = []
cal_month_non_zero = []
for i in range(1,32):
    if i < 10: 
        cal_month.append(str(i))
    elif i >= 10 and i <= 12:
        cal_month.append(str(i))  

In [4]:
GV = pd.read_csv("PPI_GV.csv")

Material_comp = pd.DataFrame()
Material_comp["time"] = GV["DATE"]
Material_comp = Material_comp.iloc[::-1]
Material_comp = Material_comp.iloc[:393]
Material_comp = Material_comp.iloc[::-1]
Material_comp = Material_comp.reset_index(drop=True)

GV = GV.drop(["DATE"], axis = 1) 
GV = GV.iloc[::-1]
GV = GV.iloc[:393]
GV = GV.iloc[::-1]
GV = GV.reset_index(drop=True)

CB = pd.read_csv("PPI_CB.csv")
CB = CB.drop(["DATE"], axis = 1) 
CB = CB.iloc[::-1]
CB = CB.iloc[:393]
CB = CB.iloc[::-1]
CB = CB.reset_index(drop=True) 

METAL = pd.read_csv("PPI_METAL.csv")
METAL = METAL.drop(["DATE"], axis = 1) 
METAL = METAL.iloc[::-1]
METAL = METAL.iloc[:393]
METAL = METAL.iloc[::-1]
METAL = METAL.reset_index(drop=True) 

CNC = pd.read_csv("PPI_CNC.csv")
CNC = CNC.drop(["DATE"], axis = 1) 
CNC = CNC.iloc[::-1]
CNC = CNC.iloc[:393]
CNC = CNC.iloc[::-1]
CNC = CNC.reset_index(drop=True) 

PL = pd.read_csv("PPI_PL.csv")
PL = PL.drop(["DATE"], axis = 1) 
PL = PL.iloc[::-1]
PL = PL.iloc[:393]
PL = PL.iloc[::-1]
PL = PL.reset_index(drop=True)

In [5]:
Material_comp["Gravel"] = GV
Material_comp["Conc_Brick"] = CB
Material_comp["Metals"] = METAL
Material_comp["Concrete_Mix"] = CNC
Material_comp["Plywood"] = PL

In [6]:
scaler = Scaler()
series = scaler.fit_transform(TimeSeries.from_dataframe(
            Material_comp, "time", ["Gravel","Conc_Brick","Metals","Concrete_Mix","Plywood"])).astype(np.float32)
train, val = series.split_after(pd.Timestamp("20160401"))

In [7]:
model_materials =  RNNModel(
    model="LSTM",
    hidden_dim=20,
    n_rnn_layers=10,
    dropout=0.2,
    batch_size=32,
    n_epochs=50,
    optimizer_kwargs={"lr": 1e-4},
    random_state=0,
    training_length=24,
    input_chunk_length=24,
    output_chunk_length=7,
    likelihood=GaussianLikelihood(),
)

In [8]:
model_materials.fit(train, val_series=val, verbose=True)

2022-10-31 13:18:35 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-10-31 13:18:35 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-10-31 13:18:35 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-10-31 13:18:35 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs
2022-10-31 13:18:35 pytorch_lightning.callbacks.model_summary INFO: 
  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 32.4 K
4 | V             | Linear           | 210   
---------------------------------------------------
32.6 K    Trainable params
0         Non-trainable params
32.6 K    Total params
0.130     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

2022-10-31 13:19:15 pytorch_lightning.utilities.rank_zero INFO: `Trainer.fit` stopped: `max_epochs=50` reached.


In [9]:
pred = model_materials.predict(n=7,series=series)

Predicting: 10it [00:00, ?it/s]

In [14]:
scaled_pred=scaler.inverse_transform(pred)
scaled_pred

<TimeSeries (DataArray) (time: 7, component: 5, sample: 1)>
array([[[187.9673 ],
        [210.90172],
        [189.33145],
        [170.23177],
        [163.01315]],

       [[166.06541],
        [ 46.89121],
        [169.84674],
        [ 75.30788],
        [147.14093]],

       [[228.33784],
        [188.44533],
        [149.30795],
        [ 87.09143],
        [128.00734]],

       [[158.0214 ],
        [103.25454],
...
        [172.77383],
        [187.566  ]],

       [[144.2032 ],
        [238.87823],
        [134.31528],
        [103.2004 ],
        [134.84259]],

       [[132.58034],
        [131.94507],
        [134.45735],
        [195.10356],
        [171.32156]],

       [[164.73315],
        [179.57373],
        [185.2551 ],
        [148.80212],
        [193.02132]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2022-10-01 2022-11-01 ... 2023-04-01
  * component  (component) object 'Gravel' 'Conc_Brick' ... 'Plywood'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [12]:
pred_out = TimeSeries.to_csv(scaled_pred,"Mat_pred_RNN.csv")

In [13]:
Material_comp

,time,Gravel,Conc_Brick,Metals,Concrete_Mix,Plywood
0,1990-01-01,123.700,122.2,119.5,110.300,114.300
1,1990-02-01,124.600,122.2,118.8,110.300,115.800
2,1990-03-01,125.100,122.2,118.8,110.900,116.100
3,1990-04-01,125.500,122.2,118.8,111.000,125.300
4,1990-05-01,126.000,122.2,118.8,111.300,122.700
...,...,...,...,...,...,...
388,2022-05-01,410.942,304.653,374.857,318.246,355.223
389,2022-06-01,421.041,309.714,378.454,322.084,347.477
390,2022-07-01,423.964,309.714,372.821,328.435,324.565
391,2022-08-01,426.322,309.714,367.714,332.472,316.151
